In [3]:
import pandas as pd
import numpy as np

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [41]:
df=pd.read_csv("train.csv")

In [45]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [53]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [55]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [57]:
x_train,x_test,y_train,y_test= train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [61]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [65]:
x_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [75]:
# create Simple Inmputer
tnf1 = ColumnTransformer([
        ("imppute_age",SimpleImputer(),[2]),
        ("impute_embarked",SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [135]:
# create OneHotEncoding
tnf2 = ColumnTransformer([
        ('ohe_sex_embarked',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
],remainder ="passthrough")

In [137]:
#Scaling
tnf3 = ColumnTransformer([
    ('scaler',MinMaxScaler(),slice(0,10))
])

In [139]:
#feature Selection
tnf4 = SelectKBest(score_func=chi2,k=10)

In [141]:
#Model Training
tnf5 = DecisionTreeClassifier()

In [143]:
# Createing Pipelines
pipe = Pipeline([
    ('tnf1',tnf1),
    ('tnf2',tnf2),
    ('tnf3',tnf3),
    ('tnf4',tnf4),
    ('tnf5',tnf5)
])

In [145]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imppute_age',
                                                  SimpleImputer(), [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(score_func=<function chi2 at 0x0000024B5FDE3880>)),
                ('tnf5', DecisionTreeClassifier())])

In [190]:
#exploring the pipeline
pipe.named_steps['tnf2'].transformers_[0][1]

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [192]:
y_predict=pipe.predict(x_test)

In [194]:
y_predict

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [198]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.6256983240223464

In [219]:
#cross validation
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train,y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

In [233]:
# gridsearchcv
params = {
    'tnf5__max_depth':[1,2,3,4,5,None]
}

In [235]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tnf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imppute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('tnf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('tnf3',
                                        ColumnTransformer(transformers=[('scaler',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('tnf4',
                                        SelectKBest(score_func=<function chi2 at 0x0000024B5FDE3880>)),
                                       ('tnf5', DecisionTreeClassifier())]),
             param_grid={'tnf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [237]:
grid.best_score_


0.6391214419383433

In [239]:
# exporting the pipeline
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))